In [1]:
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
import re
from parsivar import POSTagger

# read model

In [2]:
model = Word2Vec.load("word2vec.model")

In [3]:
list(model.wv.vocab)

['سلام',
 'با',
 'توجه',
 'به',
 'قیمت',
 'و',
 'انتظاراتی',
 'که',
 'از',
 'یک',
 'هتل',
 '5',
 'ستاره',
 'داریم',
 'اصلا',
 'ابدا',
 'خوب',
 'نبود',
 'نظافت',
 'اطاقها',
 'در',
 'حد',
 'سه',
 'ستاره\u200cهای',
 'دبی',
 'بود',
 'غذا',
 'خیلی',
 'گران',
 'سرویس',
 'اتاق',
 'آنقدر',
 'تاخیر',
 'می\u200cآمد',
 'مجبور\u200cبودیم',
 'سفارش',
 'را',
 'لغو',
 'کنیم',
 'ظروف',
 'رستوران',
 'قدیمی',
 'کدر',
 'آکسسوار',
 'جدیدها',
 'هم',
 'کیفیت',
 'پائینی',
 'برخوردار',
 'اگر',
 'انعام',
 'می\u200cدادیم',
 'رفتار',
 'پرسنل',
 'غیر',
 'اینصورت',
 'رفتارشان',
 'بدوی',
 'روستایی',
 'مآبانه',
 'ایتالیایی',
 'دارای',
 'کارکنان',
 'نیمه',
 'حرفه',
 'ایی',
 'تا',
 'حدی',
 'قابل',
 'قبول',
 'صورت',
 'اسمی',
 'خانه\u200cداری',
 '24',
 'ساعته',
 'ساعت',
 '9',
 'شب',
 'درخواست',
 'آب',
 'معدنی',
 'کردم',
 '11',
 'آوردند',
 'من',
 'نپذیرفتم',
 'ولی',
 'موقع',
 'تسویه',
 'حساب',
 'کرده',
 'بودند',
 '100',
 'درصد',
 'مطمئن',
 'هستم',
 'واقعا',
 'بخش',
 'خصوصی',
 'واگذار',
 'کنند',
 'ارائه',
 'خدمات',
 'نحو'

In [4]:
model.wv.most_similar('رستوران')

[('فیروزه', 0.6045783162117004),
 ('فود', 0.5592269897460938),
 ('فست', 0.5444839000701904),
 ('نمی\u200cرسیدیم', 0.5343543291091919),
 ('حضرتی', 0.5309603810310364),
 ('بی\u200cنهایت', 0.5183838605880737),
 ('ناهار', 0.5155868530273438),
 ('سازماندهی', 0.5124387741088867),
 ('همین\u200cها', 0.5102947354316711),
 ('صبحانه', 0.5014309287071228)]

In [5]:
w = 'شد'
model.wv.most_similar(w)[0][0]

'گردید'

# preprocess

In [6]:
from parsivar import Normalizer
my_normalizer = Normalizer()

In [7]:
def normalize(txt):
    tmp = re.sub(r'[?|$|.|!|-|_|:|,|،|)|(]', r'', txt)
    return my_normalizer.normalize(tmp)

In [8]:
from parsivar import Tokenizer
my_tokenizer = Tokenizer()

In [9]:
def divide_to_tokens(tmp_text):
    words = my_tokenizer.tokenize_words(my_normalizer.normalize(tmp_text))
    return words

In [10]:
def divide_to_sents(tmp_text):
    sents = my_tokenizer.tokenize_sentences(my_normalizer.normalize(tmp_text))
    return sents

In [11]:
my_tagger = POSTagger(tagging_model="wapiti")
my_tokenizer = Tokenizer()

In [12]:
def get_most_similar(word):
    answer = ''
    w_pos = my_tagger.parse(my_tokenizer.tokenize_words(word))[0][1]
    for i in range(10):
        t = str(model.wv.most_similar(word)[i][0])
        t_pos = my_tagger.parse(my_tokenizer.tokenize_words(t))[0][1]
        if(w_pos == t_pos):
            return t
    return str(model.wv.most_similar(word)[0][0])

In [13]:
sw = ['و',
'برای',
'یا',
'پس',
'اگر',
'نه',
'را',
'چون',
'چه',
'تا',
'اما',
'باری',
'خواه',
'زیرا',
'که',
'لیکن',
'نیز',
'ولی',
'هم',
'خواه',
'چه',
'یا',
'نه',
'هم',
'بلکه',
'چنانچه',
'اگرچه']

# test

In [14]:
tmp = "اتاق خیلی خوب است. و برای رفتار پرسنل بد شد "

In [15]:
sents = divide_to_sents(tmp)
sents

['اتاق خیلی خوب است  .', ' و برای رفتار پرسنل بد شد']

In [16]:
new_txt = []
for s in sents:
    new_s = ''
    words = divide_to_tokens(s)
    for w in words:
        if(w not in sw):
            try:
                new_s = new_s + ' ' + get_most_similar(w)
            except:
                new_s = new_s + ' ' + w
        else:
            new_s = new_s + ' ' + w
    new_txt.append(new_s)
final_txt = ' '.join(new_txt)

In [17]:
print(final_txt)

 اتاق‌‌ها بسیار درسطح می‌خورد .  و برای برخورد کارکنان نامناسب گردید
